In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
def getListTask(folder_name, label):
  df = pd.DataFrame() 
  for root, dir, file in os.walk(folder_name):
    for f in file:
      if '.csv' in f:
        print(os.path.join(root, f))
        df = pd.concat([df, pd.read_csv(os.path.join(root, f))], sort=False)
    # df = df.loc[df["label"] != 0]
    data = list(set(df["task"].to_list())) 
    labels = [label for i in range(len(data))]
    return df, pd.DataFrame({"task":data, "label": labels}, columns= ["task", "label"])  

# df = getListTask("/content/drive/My Drive/Data_2021/Data_9-2-2021/data/nor")

In [ ]:
# from pandas import read_csv 
# df = read_csv("/content/drive/My Drive/Data_2021/Data_9-2-2021/data/susp/Suci.csv")

In [ ]:
process_data = pd.DataFrame() 
task_data = pd.DataFrame() 
label = {"nor": 1,"mal" : 3, "susp" : 2 }
for root, dir, file in os.walk("/content/drive/My Drive/Data_2021/Data_9-2-2021/data"):
  for d in dir:
    # print(os.path.join(root, d))
    df_process, df_task =  getListTask(os.path.join(root, d),label[d] )
    process_data = pd.concat([process_data, df_process], sort=False)
    task_data = pd.concat([task_data, df_task], sort=False)
print(process_data.shape[0], task_data.shape[0])
# process_data = process_data.drop(['task', 'CreationTimestamp', 'ProcessID', 'ParentPID', 'CommandLine', 'Image'] , axis = 1)
process_data["Scores_Network"] = process_data["Scores_Network"].astype(int)
process_data["Autostart"] = process_data["Autostart"].astype(int)
process_data["LowAccess"] = process_data["LowAccess"].astype(int)
process_data["ProcessType"] = process_data["ProcessType"].astype("category").cat.codes + 1
process_data["FileType"] = process_data["FileType"].astype("category").cat.codes + 1
process_data = process_data.drop(["Unnamed: 45"], axis=1)

/content/drive/My Drive/Data_2021/Data_9-2-2021/data/mal/Mal_1.csv
/content/drive/My Drive/Data_2021/Data_9-2-2021/data/mal/Mal_2.csv
/content/drive/My Drive/Data_2021/Data_9-2-2021/data/mal/Mal_3.csv
/content/drive/My Drive/Data_2021/Data_9-2-2021/data/mal/Mal_4.csv
/content/drive/My Drive/Data_2021/Data_9-2-2021/data/mal/Mal_5.csv
/content/drive/My Drive/Data_2021/Data_9-2-2021/data/mal/Mal_Minh.csv
/content/drive/My Drive/Data_2021/Data_9-2-2021/data/susp/Suci.csv
/content/drive/My Drive/Data_2021/Data_9-2-2021/data/nor/Normal_Hoa.csv
/content/drive/My Drive/Data_2021/Data_9-2-2021/data/nor/Normal_Hu.csv
820455 50985


In [ ]:
data = process_data[process_data['task'].isin(task_data[task_data["label"] == 2]["task"].tolist())]
data.isnull().sum()

task                               0
CreationTimestamp                  0
ProcessID                          0
ParentPID                          0
CommandLine                        5
Image                              2
ProcessType                        0
Registry                           0
Files                              0
Modules                            0
DroppedFiles                       0
DebugStrings                       0
EventsCounters_Network             0
Scores_Network                     0
Autostart                          0
LowAccess                          0
FileType                           0
Priority                           0
unknown_ip                         0
Suspicious_ip                      0
Malicious_ip                       0
whitelist_ip                       0
unsafe_ip                          0
unknown_domain                     0
Suspicious_domain                  0
Malicious_domain                   0
whitelist_domain                   0
u

In [ ]:
import math
# data = df.drop(['task', 'CreationTimestamp', 'ProcessID', 'ParentPID', 'CommandLine', 'Image'] , axis = 1)
# data["Scores_Network"] = data["Scores_Network"].astype(int)
# data["Autostart"] = data["Autostart"].astype(int)
# data["LowAccess"] = data["LowAccess"].astype(int)
# data["ProcessType"] = data["ProcessType"].astype("category").cat.codes + 1
# data["FileType"] = data["FileType"].astype("category").cat.codes + 1
print(data.shape[0])
data = data[:][:]
data_copy = data.drop(['task', 'CreationTimestamp', 'ProcessID', 'ParentPID', 'CommandLine', 'Image'] , axis = 1)

# data = df.drop(['task', 'CreationTimestamp', 'ProcessID', 'ParentPID', 'CommandLine', 'Image'] , axis = 1)
print(data_copy.shape[0])
data_copy.loc[data_copy['label'] == 0, 'label'] = math.nan
print(data_copy[data_copy['label']==0].shape[0])
print(data[data['label'] == 0].shape[0])
data.sample(5)

36457
36457
0
20344


,task,CreationTimestamp,ProcessID,ParentPID,CommandLine,Image,ProcessType,Registry,Files,Modules,DroppedFiles,DebugStrings,EventsCounters_Network,Scores_Network,Autostart,LowAccess,FileType,Priority,unknown_ip,Suspicious_ip,Malicious_ip,whitelist_ip,unsafe_ip,unknown_domain,Suspicious_domain,Malicious_domain,whitelist_domain,unsafe_domain,unknown_http_request,Suspicious_http_request,Malicious_http_request,whitelist_http_request,unsafe_http_request,HKEY_LOCAL_MACHINE,HKEY_CLASSES_ROOT,HKEY_USERS,HKEY_CURRENT_CONFIG,HKEY_CURRENT_USER,startup_registry_keys,active_setup_registry_keys,services_registry_keys,dll_injection_registry_keys,shell_spawning_registry_keys,internet_settings_registry_keys,bho_registry_keys,label
33104,b69a0442-7773-4e34-b04a-027c89056d2b,1611659328269,3224,3288,"""C:\Program Files\Google\Chrome\Application\ch...",C:\Program Files\Google\Chrome\Application\chr...,1,0,1002,118,46,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
24012,745a13f8-306f-48d2-8358-8cae05ade864,1607195158833,3064,1532,C:\Windows\system32\cmd.exe /K powershell -n...,C:\Windows\system32\cmd.exe,1,6,218,16,0,0,0,0,0,0,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
31224,7437d4ac-a4bb-42ef-aad8-4a2f41f04be4,1606762572440,2348,376,"""C:\PROGRA~1\MICROS~1\Office14\OUTLOOK.EXE"" /e...",C:\PROGRA~1\MICROS~1\Office14\OUTLOOK.EXE,6,1827,5346,225,32,0,4,1,0,0,10,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,7,6,0,0,57,0,0,0,0,0,0,0,2
23903,6d092ba9-26fc-4ac7-959c-6d04049b0c4b,1610430847593,1004,732,"""C:\Program Files\Google\Chrome\Application\ch...",C:\Program Files\Google\Chrome\Application\chr...,1,0,320,118,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10052,88033a75-ec63-4b06-8ccf-356c6e9a0e29,1609404203657,2120,2184,"""C:\Program Files\Google\Chrome\Application\ch...",C:\Program Files\Google\Chrome\Application\chr...,1,0,318,118,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
data_copy.isnull().sum()

ProcessType                            0
Registry                               0
Files                                  0
Modules                                0
DroppedFiles                           0
DebugStrings                           0
EventsCounters_Network                 0
Scores_Network                         0
Autostart                              0
LowAccess                              0
FileType                               0
Priority                               0
unknown_ip                             0
Suspicious_ip                          0
Malicious_ip                           0
whitelist_ip                           0
unsafe_ip                              0
unknown_domain                         0
Suspicious_domain                      0
Malicious_domain                       0
whitelist_domain                       0
unsafe_domain                          0
unknown_http_request                   0
Suspicious_http_request                0
Malicious_http_r

In [ ]:
from sklearn.impute import KNNImputer
from numpy import isnan
import pandas as pd

# print total missing
print('Missing: %d' % data_copy.isnull().sum().sum())
print(data_copy[data_copy['label']==0].shape[0])
# define imputer
imputer = KNNImputer(n_neighbors=10)
# fit on the dataset
imputer.fit(data_copy)
# transform the dataset
Xtrans = imputer.transform(data_copy)
# print total missing
print('Missing: %d' % sum(isnan(Xtrans).flatten()))
# out = pd.DataFrame(Xtrans).round(0)

Missing: 20344
0
Missing: 0


In [ ]:
out = data.assign(label=Xtrans[:,-1]).round(0)
out.sample(5)
out["bho_registry_keys"] = out["bho_registry_keys"].astype(int)
out.to_csv("/content/drive/My Drive/Data_2021/Data_9-2-2021/imputed_data/susp/imputed_susp_data.csv")
# out.to_csv("/content/drive/My Drive/Data_2021/Data_9-2-2021/imputed_data/nor/imputed_susp_data.csv", index = False)
# print('Missing: %d' % sum(isnan(Xtrans).flatten()))
# numbers = df2["Numbers"]
# df1 = df1.join(numbers)


In [ ]:
out.isnull().sum()

task                               0
CreationTimestamp                  0
ProcessID                          0
ParentPID                          0
CommandLine                        5
Image                              2
ProcessType                        0
Registry                           0
Files                              0
Modules                            0
DroppedFiles                       0
DebugStrings                       0
EventsCounters_Network             0
Scores_Network                     0
Autostart                          0
LowAccess                          0
FileType                           0
Priority                           0
unknown_ip                         0
Suspicious_ip                      0
Malicious_ip                       0
whitelist_ip                       0
unsafe_ip                          0
unknown_domain                     0
Suspicious_domain                  0
Malicious_domain                   0
whitelist_domain                   0
u

In [ ]:
import numpy as np

# print(out)
out_cop = out.round(0)
# print(out_cop)
data_copy = data[:408110][:]
data_copy = data_copy.reset_index(drop = True)
# print(data_copy)
data_copy['label2'] = out_cop[out_cop.columns[-1]]

data_cop = data_copy[data_copy['label'] != 0]
print("Label != 0: ", data_cop.shape[0])
filter0 = data_copy['label'] == 0
filter1 = data_copy['label2'] == 1.0
filter2 = data_copy['label2'] == 2.0
filter3 = data_copy['label2'] == 3.0

df1 =  data_copy.where(filter0 & filter1, inplace = False).dropna()
df2 =  data_copy.where(filter0 & filter2, inplace = False).dropna()
df3 =  data_copy.where(filter0 & filter3, inplace = False).dropna()
print("Label 1=>0: ",df1.shape[0], "\t", df1.shape[0]/408110 *100)
print("Label 2=>0: ", df2.shape[0], "\t", df2.shape[0]/408110 *100)
print("Label 3=>0: ", df2.shape[0], "\t", df3.shape[0]/408110 *100)
print("Total: ", data_cop.shape[0]+df1.shape[0]+df2.shape[0]+df3.shape[0])

Label != 0:  213684
Lable 1=>0:  5880 	 1.4407880228369803
Lable 2=>0:  33707 	 8.259292837715321
Lable 3=>0:  33707 	 37.94050623606381
Total:  408110


In [ ]:
df = pd.read_csv("/content/drive/My Drive/Data_2021/Data_9-2-2021/imputed_data/susp/imputed_susp_data.csv")
df.isnull().sum()

Unnamed: 0                         125
task                                 0
CreationTimestamp                    0
ProcessID                            0
ParentPID                            0
CommandLine                          5
Image                              127
ProcessType                        125
Registry                           125
Files                              125
Modules                            125
DroppedFiles                       125
DebugStrings                       125
EventsCounters_Network             125
Scores_Network                     125
Autostart                          125
LowAccess                          125
FileType                           125
Priority                           125
unknown_ip                         125
Suspicious_ip                      125
Malicious_ip                       125
whitelist_ip                       125
unsafe_ip                          125
unknown_domain                     125
Suspicious_domain        